In [93]:
import chess
import chess.svg, chess.pgn
import random
import time
from IPython.display import clear_output, SVG, display_svg, display_html
import collections
import math
import cProfile

In [94]:
board = chess.Board()
board.reset()

In [95]:

def makeRandomMove(board):
    index = random.randint(0, len(list(board.legal_moves)) - 1)
    move_uci = list(board.legal_moves)[index]
    #print(move_uci)
    board.push(move_uci)
    return


In [96]:
def makeRandomCapture(board):
    moves = list(board.legal_moves)
    captures = [move for move in moves if board.is_capture(move)]
    if(captures == []):
        index = random.randint(0, len(moves) - 1)
        board.push(moves[index])

    else:
        index = random.randint(0, len(captures) - 1)
        board.push(captures[index])

    return

In [97]:
def board_to_game(board):
    game = chess.pgn.Game()

    # Undo all moves.
    switchyard = collections.deque()
    while board.move_stack:
        switchyard.append(board.pop())

    game.setup(board)
    node = game

    # Replay all moves.
    while switchyard:
        move = switchyard.pop()
        node = node.add_variation(move)
        board.push(move)

    game.headers["Result"] = board.result()
    return game

    #https://github.com/niklasf/python-chess/issues/63

In [98]:
# Returns an int - positive means white better, negative means black better, 0 is equal
# Simply adds up the piece values from FEN

piece_values = {
    "p" : -1,
    "P" : 1,
    "r" : -5,
    "R" : 5,
    "n" : -3,
    "N" : 3,
    "b" : -3,
    "B" : 3,
    "q" : -9,
    "Q" : 9
}

def evaluate_position(board):

    if(board.outcome() == None):
        fen_string = board.board_fen()
        eval = 0

        for char in fen_string:
            eval += piece_values[char] if char in piece_values else 0

        return eval
    
    else:
        result = board.outcome().result()

        if(result == "1-0"):
            #print("Win for white")
            return 300
            
        elif(result == "0-1"):
            return -300

        else:
            return 0

    
        

In [99]:
evaluate_position(board)

0

In [100]:
# Level 1 AI - makes random moves

def random_AI_game(board, display=False):
    while(True):
        makeRandomMove(board)

        if not (board.outcome() == None):
            print("Game Over")
            break

        if(display):    
            display_svg(chess.svg.board(board, size=700))
            clear_output(wait=True)
        

        time.sleep(0.01)

    print(board_to_game(board))

    return

random_AI_game(board)

Game Over
[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "1/2-1/2"]

1. Nh3 a6 2. a3 d6 3. g3 h5 4. d4 a5 5. Nf4 g5 6. Nh3 Qd7 7. Kd2 e5 8. Ke1 c5 9. Qd3 b6 10. Nxg5 Qa7 11. Nxf7 Kd7 12. Nh6 Rh7 13. Qe4 a4 14. Qg4+ Kc6 15. Qf5 h4 16. Qf3+ Kc7 17. Nf7 Bg7 18. Qc6+ Nxc6 19. Bf4 Na5 20. b4 Rb8 21. c4 Rh8 22. Kd1 e4 23. Bxd6+ Kb7 24. Bf4 Qa8 25. Kd2 Nh6 26. Nd6+ Kc7 27. Nc3 cxd4 28. Nd5+ Qxd5 29. Nb7+ Qe5 30. Rb1 Bxb7 31. Ra1 Kd7 32. gxh4 Rhd8 33. Bg2 Rh8 34. Kd1 Qe6 35. Bg5 d3 36. Kd2 Bc8 37. Kc1 Ba6 38. Ra2 Qf6 39. e3 Nf5 40. f3 Rbf8 41. Rb2 Nb3+ 42. Kb1 Qc3 43. f4 Nc1 44. Bd8 Rxh4 45. Rb3 Rf6 46. h3 Rd6 47. c5 Rg4 48. hxg4 Bb5 49. Bxb6 Ke6 50. Rg1 Be5 51. Bc7 Na2 52. Bf3 Rd4 53. Ba5 Nd6 54. Bc7 Nf5 55. Re1 Ke7 56. Bh1 Be8 57. Rg1 Bxc7 58. Bxe4 Bb8 59. Bh1 Qb2+ 60. Rxb2 Rd7 61. g5 Nh6 62. Bb7 Rd5 63. Rgg2 Nf5 64. Kxa2 Bxf4 65. e4 d2 66. Bc8 Bb8 67. Rb1 Bc7 68. b5 Ng7 69. Rf1 Rd3 70. Rfg1 Rd5 71. Bh3 Bg6 72. Rg4 Kf8 73. Ra1 Ba5 74. Rd1 Kg8 75. Rxd2 

In [101]:
# Level 2 AI - prioritises captures

def capture_AI_game(board, display=False):
    while(True):

        # Choose captures
        makeRandomCapture(board)

        if not (board.outcome() == None):
            print("Game Over")
            break

        if(display):    
            display_svg(chess.svg.board(board, size=700))
            clear_output(wait=True)
        

        #time.sleep(0.01)

    print(board_to_game(board))

    return

In [102]:
capture_AI_game(board, True)

ValueError: empty range for randrange() (0,0, 0)

In [103]:
# Performs a minimax search and returns best move, along with the evaluation

def minimax_search(board, depth, white_to_move):

    if(depth == 0 or board.is_game_over()):
        eval = evaluate_position(board)
        return (None,eval)

    else:
        moves_list = list(board.legal_moves)
        random.shuffle(moves_list)

        best_eval = -math.inf if white_to_move else math.inf
        best_move = ''
        
        for move in moves_list:
            # Evaluate each move and recursively find the best one

            board.push(move)
            current_eval = minimax_search(board, depth - 1, not white_to_move)[1]

            if(white_to_move):
                if(current_eval >= best_eval):
                    best_eval = current_eval
                    best_move = move


            else:
                if(current_eval <= best_eval):
                    best_eval = current_eval
                    best_move = move

            board.pop()

        return (best_move, best_eval)

In [104]:
# Level 3 AI - minimax bots playing against each other

def minimax_game(board, depth, display=False):

    board.reset()

    white_to_move = True
    
    while(True):

        search = minimax_search(board, depth, white_to_move)
        #print(search)
        board.push(search[0])
        white_to_move = not white_to_move

        if not (board.outcome() == None):
            print("Game Over")
            break

        if(display):    
            display_svg(chess.svg.board(board, size=700))
            clear_output(wait=True)
        

        #time.sleep(0.01)

    print(board_to_game(board))

    return

In [ ]:
minimax_game(board, 3, True)

(Move.from_uci('e8f8'), inf)
Game Over
[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "1-0"]

1. Nf3 e6 2. Ne5 Qh4 3. a4 Qe4 4. Nd3 Nc6 5. c3 Nf6 6. Qb3 a6 7. Ra2 Ng8 8. f4 d6 9. Kf2 Nce7 10. Ke1 Nf6 11. e3 c6 12. a5 g6 13. Ra3 Nd7 14. Rg1 d5 15. Be2 Rg8 16. Kf1 Ra7 17. Ke1 Nf5 18. Ra1 Rg7 19. Nf2 Nd4 20. cxd4 Qf5 21. Nc3 Nb8 22. Nfd1 Be7 23. Nf2 f6 24. g4 Qe4 25. Bd1 Ba3 26. Rxa3 Rd7 27. Qb6 h6 28. Qxa7 b5 29. Qxb8 Rb7 30. Qxc8+ Ke7 31. Qxb7+ Kd8 32. Qb8+ Kd7 33. Qa7+ Kd8 34. Qxa6 b4 35. Qb6+ Kd7 36. Qxb4 c5 37. Qb5+ Kd8 38. dxc5 Qc4 39. Qc6 f5 40. Ra4 Qxa4 41. Bxa4 Ke7 42. Qe8+ Kf6 43. Qf8# 1-0


In [105]:
def move_ordering_sort(board, only_captures = False):

    moves_list = list(board.legal_moves)

    captures = []
    #checks = []
    temp_list = []

    for move in moves_list:
        if(board.is_capture(move)):
            captures.append(move)

        # elif(board.gives_check(move)):
        #     checks.append(move)

        else:
            temp_list.append(move)

    if(only_captures):
        return captures

    else:
        return captures + temp_list



In [118]:
total_branches_pruned = 0
total_quiescence_positions_evaluated = 0
total_quiescence_calls = 0

def quiescence_search(board, white_to_move, alpha, beta, depth):

    global total_quiescence_calls
    total_quiescence_calls += 1

    # Very similar to alpha beta - but we only look at captures and checkmates - to solve the horizon issue without huge branching

    moves_list = move_ordering_sort(board, only_captures=True)

    #static_eval = evaluate_position(board)

    # if(white_to_move and beta <= static_eval):
    #     return (None, static_eval)

    # elif((not white_to_move) and alpha >= static_eval):
    #     return (None, static_eval)

    #print(moves_list)

    #moves_list = list(board.legal_moves)

    if(board.is_game_over() or moves_list == [] or depth == 0):
        eval = evaluate_position(board)
        global total_quiescence_positions_evaluated
        total_quiescence_positions_evaluated += 1
        return (None,eval)

    else:
        #random.shuffle(moves_list)
        best_eval = -math.inf if white_to_move else math.inf
        best_move = ''
        global total_branches_pruned

        if(white_to_move):               
            for move in moves_list:
                # Evaluate each move and recursively find the best one

                board.push(move)
                current_eval = quiescence_search(board, False, alpha, beta, depth - 1)[1]  
                if(current_eval > best_eval):
                    best_eval = current_eval
                    best_move = move

                board.pop()

                alpha = max(alpha, current_eval)

                #if(alpha > 100): print(depth, move, alpha, beta)

                if(alpha >= beta):

                    total_branches_pruned += 1
                    break


        else:
            for move in moves_list:
                # Evaluate each move and recursively find the best one

                board.push(move)
                current_eval = quiescence_search(board, True, alpha, beta, depth - 1)[1]  
                if(current_eval < best_eval):
                    best_eval = current_eval
                    best_move = move

                board.pop()

                beta = min(beta, current_eval)

                if(alpha >= beta):

                    total_branches_pruned += 1
                    break

        #print(depth, "Best move is: " + str(best_move), "Eval: " + str(best_eval), end=" ")
        return (best_move, best_eval)

In [119]:
# Performs a minimax search with alpha/beta pruning and returns best move, along with the evaluation
total_alphabeta_nodes_searched = 0
total_alphabeta_calls = 0

def alphabeta_search(board, depth, white_to_move, alpha, beta):

    #moves_list = list(board.legal_moves)
    global total_alphabeta_calls
    total_alphabeta_calls += 1

    if(depth == 0 or board.is_game_over()):
        global total_alphabeta_nodes_searched
        total_alphabeta_nodes_searched += 1
        eval = quiescence_search(board, white_to_move, alpha, beta, 99)[1]
        #eval = evaluate_position(board)
        return (None,eval)

    else:      
        moves_list = move_ordering_sort(board, only_captures=False)
        random.shuffle(moves_list)
        
        best_eval = -math.inf if white_to_move else math.inf
        best_move = ''
        global total_branches_pruned

        if(white_to_move):               
            for move in moves_list:
                # Evaluate each move and recursively find the best one

                board.push(move)
                current_eval = alphabeta_search(board, depth - 1, False, alpha, beta)[1]  
                if(current_eval > best_eval):
                    best_eval = current_eval
                    best_move = move

                board.pop()

                alpha = max(alpha, current_eval)

                #if(alpha > 100): print(depth, move, alpha, beta)

                if(alpha >= beta):

                    total_branches_pruned += 1
                    break


        else:
            for move in moves_list:
                # Evaluate each move and recursively find the best one

                board.push(move)
                current_eval = alphabeta_search(board, depth - 1, True, alpha, beta)[1]  
                if(current_eval < best_eval):
                    best_eval = current_eval
                    best_move = move

                board.pop()

                beta = min(beta, current_eval)

                if(alpha >= beta): 

                    total_branches_pruned += 1
                    break

        #print(depth, "Best move is: " + str(best_move), "Eval: " + str(best_eval), end=" ")
        return (best_move, best_eval)

In [130]:
def reset_stat_variables():
    global total_quiescence_positions_evaluated

    global total_quiescence_calls

    global total_alphabeta_nodes_searched

    global total_alphabeta_calls

    global total_branches_pruned


    total_quiescence_positions_evaluated = 0
    total_quiescence_calls = 0
    total_alphabeta_nodes_searched = 0
    total_alphabeta_calls = 0
    total_branches_pruned = 0


def print_stat_variables():

    global total_quiescence_positions_evaluated

    global total_quiescence_calls

    global total_alphabeta_nodes_searched

    global total_alphabeta_calls

    global total_branches_pruned

    print("Quiescent Nodes(leaves) evaluated:", total_quiescence_positions_evaluated)
    print("Total quiescence function calls(recursive) for last move:", total_quiescence_calls)
    print("Alphabeta Nodes(leaves) reached:", total_alphabeta_nodes_searched)
    print("Alphabeta function calls(recursive):", total_alphabeta_calls)
    print("Total branches pruned this step:", total_branches_pruned)


reset_stat_variables()

In [131]:
# Level 4 AI - minimax bots with alpha-beta pruning

def alphabeta_game(board, depth, display=False):

    board.reset()
    white_to_move = True
    
    while(True):
        
        reset_stat_variables()
        board.push(alphabeta_search(board, depth, white_to_move, -math.inf, math.inf)[0])
        white_to_move = not white_to_move

        print_stat_variables()

        
        if not (board.outcome() == None):
            print("Game Over")
            break

        if(display):    
            display_svg(chess.svg.board(board, size=700))
            
        clear_output(wait=True)
        
        #time.sleep(0.01)

    print(board_to_game(board))

    return

In [132]:
#alphabeta_game(board, 3, True)

cProfile.run('alphabeta_game(board, 3, True)')

         312433066 function calls (311713804 primitive calls) in 203.060 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 <decorator-gen-2>:1(__call__)
        5    0.000    0.000    0.000    0.000 <decorator-gen-4>:1(__call__)
        5    0.000    0.000    0.000    0.000 <decorator-gen-5>:1(__call__)
       15    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
   718138   10.653    0.000  116.307    0.000 <ipython-input-105-08c859d937b6>:1(move_ordering_sort)
717216/10824    4.371    0.000  209.653    0.019 <ipython-input-118-c9a874dbabb0>:5(quiescence_search)
  11746/6    0.054    0.000  210.279   35.046 <ipython-input-119-d5013c0abb79>:5(alphabeta_search)
        6    0.000    0.000    0.000    0.000 <ipython-input-130-978580c7e881>:1(reset_stat_variables)
        5    0.000    0.000    0.002    0.000 <ipython-input-130-978580c7e881>:20(p

KeyboardInterrupt: 

In [ ]:
def test_position(board, fen, depth):
    board.set_fen(fen)
    white_to_move = True if board.turn == chess.WHITE else False
    print(alphabeta_search(board, depth, white_to_move, -math.inf, math.inf))
    return

test_position(board, "8/qQ5p/3pN2K/3pp1R1/4k3/7N/1b1PP3/8 w - - 0 1", 5)

(Move.from_uci('b7a7'), inf)


In [ ]:
board.set_fen("rnbqkbnr/pppp1ppp/8/8/8/8/PPPPQPPP/RNBQKBNR w KQkq - 0 1")
board
board.is_pseudo_legal(board.parse_san("Qxe8"))

True